<a href="https://colab.research.google.com/github/cbologa/public_databases/blob/main/R/ChEMBL_get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Latest ChEMBL database data dictionary is available at https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/schema_documentation.html

Load required packages

In [20]:
if(!require(pacman)) install.packages("pacman")
pacman::p_load(R.utils, RSQLite, curl)
library(DBI)

Download and unpack the SQLite version of ChEMBL database

In [31]:
zip  <- "chembl_31_sqlite.tar.gz"
file <- "chembl_31/chembl_31_sqlite/chembl_31.db"
if(!file.exists(file)){
   curl_download(paste0("ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/",zip),zip)
   untar(zip,files = file, exdir = "./")
}

Connect to database

In [32]:
con <- dbConnect(RSQLite::SQLite(), file)

Show all tables in the database

In [33]:
dbListTables(con)

[1] "action_type"                   "activities"                   
 [3] "activity_properties"           "activity_smid"                
 [5] "activity_stds_lookup"          "activity_supp"                
 [7] "activity_supp_map"             "assay_class_map"              
 [9] "assay_classification"          "assay_parameters"             
[11] "assay_type"                    "assays"                       
[13] "atc_classification"            "binding_sites"                
[15] "bio_component_sequences"       "bioassay_ontology"            
[17] "biotherapeutic_components"     "biotherapeutics"              
[19] "cell_dictionary"               "chembl_id_lookup"             
[21] "component_class"               "component_domains"            
[23] "component_go"                  "component_sequences"          
[25] "component_synonyms"            "compound_properties"          
[27] "compound_records"              "compound_structural_alerts"   
[29] "compound_structures"           "confidence_score_lookup"      
[31] "curation_lookup"               "data_validity_lookup"         
[33] "defined_daily_dose"            "docs"                         
[35] "domains"                       "drug_indication"              
[37] "drug_mechanism"                "drug_warning"                 
[39] "formulations"                  "frac_classification"          
[41] "go_classification"             "hrac_classification"          
[43] "indication_refs"               "irac_classification"          
[45] "ligand_eff"                    "mechanism_refs"               
[47] "metabolism"                    "metabolism_refs"              
[49] "molecule_atc_classification"   "molecule_dictionary"          
[51] "molecule_frac_classification"  "molecule_hierarchy"           
[53] "molecule_hrac_classification"  "molecule_irac_classification" 
[55] "molecule_synonyms"             "organism_class"               
[57] "patent_use_codes"              "predicted_binding_domains"    
[59] "product_patents"               "products"                     
[61] "protein_class_synonyms"        "protein_classification"       
[63] "protein_family_classification" "relationship_type"            
[65] "research_companies"            "research_stem"                
[67] "site_components"               "source"                       
[69] "sqlite_stat1"                  "structural_alert_sets"        
[71] "structural_alerts"             "target_components"            
[73] "target_dictionary"             "target_relations"             
[75] "target_type"                   "tissue_dictionary"            
[77] "usan_stems"                    "variant_sequences"            
[79] "version"                       "warning_refs"

Show fields in a table/view

In [34]:
dbListFields(con, "activities")

[1] "activity_id"           "assay_id"              "doc_id"               
 [4] "record_id"             "molregno"              "standard_relation"    
 [7] "standard_value"        "standard_units"        "standard_flag"        
[10] "standard_type"         "activity_comment"      "data_validity_comment"
[13] "potential_duplicate"   "pchembl_value"         "bao_endpoint"         
[16] "uo_units"              "qudt_units"            "toid"                 
[19] "upper_value"           "standard_upper_value"  "src_id"               
[22] "type"                  "relation"              "value"                
[25] "units"                 "text_value"            "standard_text_value"

Extract references, bioactivities, and compounds tested on equine encephalitis viruses

In [35]:
df1 <- dbGetQuery(con, "
select
  target_dictionary.*,
  assays.*,
  activities.*,
  compound_structures.standard_inchi_key,
  compound_structures.canonical_smiles,
  molecule_dictionary.*,
  docs.*
from
  target_dictionary
  left join assays on target_dictionary.tid=assays.tid
  left join docs on assays.doc_id=docs.doc_id
  left join activities on assays.assay_id=activities.assay_id
  left join molecule_dictionary on activities.molregno=molecule_dictionary.molregno
  left join compound_structures on molecule_dictionary.molregno=compound_structures.molregno
where 
  target_dictionary.organism like '%equine encephalitis virus'
group by
  target_dictionary.tid,
  assays.assay_id,
  activities.activity_id,
  compound_structures.standard_inchi_key,
  compound_structures.canonical_smiles,
  molecule_dictionary.molregno,
  docs.doc_id
order by
  docs.year,
  docs.doi
")

#save results to file
write.csv(df1, "chembl_eev.csv", row.names=FALSE)

# Show results
df1

tid,target_type,pref_name,tax_id,organism,chembl_id,species_group_flag,assay_id,doc_id,description,⋯,pubmed_id,doi,chembl_id,title,doc_type,authors,abstract,patent_id,ridx,src_id
<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<int>,<chr>,⋯,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
50637,ORGANISM,Venezuelan equine encephalitis virus,11036,Venezuelan equine encephalitis virus,CHEMBL613726,0,1359874,83427,Antiviral activity against Venezuelan equine encephalitis virus infected in po dosed monkey assessed as reduction in viremia,⋯,191610,10.1021/jm00213a011,CHEMBL3272338,"Bis-basic-substituted polycyclic aromatic compounds. A new class of antiviral agents. 8. Bis-basic derivatives of carbazole, dibenzofuran, and dibenzothiophene.",PUBLICATION,"Albrecht WL, Fleming RW, Horgan SW, Mayer GD.","A series of bisalkamine esters, bis-basic ethers, and bis-basic ketones of carbazole, N-ethylcarbazole, dibenzofuran, and dibenzothiophene was synthesized and evaluated for antiviral activity. The series also included two bis-basic alkanes of N-ethylcarbazole and one bis-basic carboxamide of dibenzofuran. Structure-activity relationships indicated that within the carbazole and N-ethylcarbazole series the bisalkamine esters gave the most active derivatives while the bis-basic ketone derivatives of dibenzofuran and dibenzothiophene afforded the more potent compounds within the respective series. The [6,5,6]heterocyclic nuclei were compared with the [6,5,6] aromatic nuclei (fluroene and fluoren-9-one) including tilorone with respect to antiviral activity against encephalomyocarditis (EMC) virus. Maximum activity was associated with the bis-basic ketone side chain and fluoren-9-one nucleus.",NA,CLD0,1
50637,ORGANISM,Venezuelan equine encephalitis virus,11036,Venezuelan equine encephalitis virus,CHEMBL613726,0,1359874,83427,Antiviral activity against Venezuelan equine encephalitis virus infected in po dosed monkey assessed as reduction in viremia,⋯,191610,10.1021/jm00213a011,CHEMBL3272338,"Bis-basic-substituted polycyclic aromatic compounds. A new class of antiviral agents. 8. Bis-basic derivatives of carbazole, dibenzofuran, and dibenzothiophene.",PUBLICATION,"Albrecht WL, Fleming RW, Horgan SW, Mayer GD.","A series of bisalkamine esters, bis-basic ethers, and bis-basic ketones of carbazole, N-ethylcarbazole, dibenzofuran, and dibenzothiophene was synthesized and evaluated for antiviral activity. The series also included two bis-basic alkanes of N-ethylcarbazole and one bis-basic carboxamide of dibenzofuran. Structure-activity relationships indicated that within the carbazole and N-ethylcarbazole series the bisalkamine esters gave the most active derivatives while the bis-basic ketone derivatives of dibenzofuran and dibenzothiophene afforded the more potent compounds within the respective series. The [6,5,6]heterocyclic nuclei were compared with the [6,5,6] aromatic nuclei (fluroene and fluoren-9-one) including tilorone with respect to antiviral activity against encephalomyocarditis (EMC) virus. Maximum activity was associated with the bis-basic ketone side chain and fluoren-9-one nucleus.",NA,CLD0,1
50637,ORGANISM,Venezuelan equine encephalitis virus,11036,Venezuelan equine encephalitis virus,CHEMBL613726,0,215195,8662,In vitro antiviral activity was tested against venezuelan equine encephalomyelitis (VEE),⋯,2993613,10.1021/jm00147a013,CHEMBL1123027,Synthesis and biological activity of 5-thiobredinin and certain related 5-substituted imidazole-4-carboxamide ribonucleosides.,PUBLICATION,"Wood SG, Upadhya KG, Dalley NK, McKernan PA, Canonico PG, Robins RK, Revankar GR.","A number of 5-substituted imidazole-4-carboxamide ribonucleosides were prepared and tested for their biological activity. Treatment of 5-chloro-1-beta-D-ribofuranosylimidazole-4-carboxamide (2) with methanethiol provided 5-(methylthio)-1-beta-D-ribofuranosylimidazole-4-carboxamide (3a). Similar treatment of 2 with ethanethiol or benzenemethanethiol gave the corres

Close database connection

In [ ]:
dbDisconnect(con)